# Modelo de Regresión Lineal

## Introducción

##### En el siguiente proyecto estaré desarrollando un modelo de machine learning para una compañia de petróleo que desea saber cual es el mejor lugar para abrir 200 nuevos pozos. Estaré estudiando los beneficios y riesgos potenciales basados en los datos sobre muestras de crudo en tres regiones distintas y el modelo deberá seleccionar la región con el mayor márgen de beneficio para la empresa.

### Tabla de contenido:

## 1. Inicialización

In [5]:
#importar librerías
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib as plt 
from sklearn.linear_model import LinearRegression


/var/folders/82/grjgs_q959j1pwfb817_nsl80000gn/T/ipykernel_5102/2248431528.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 1.2. Cargar datos

In [6]:
#cargar datasets
dataset_0 = pd.read_csv('files /data/geo_data_0.csv')
dataset_1 = pd.read_csv('files /data/geo_data_1.csv')
dataset_2 = pd.read_csv('files /data/geo_data_2.csv')

### 1.2.1. Preparar Dataset 0

In [7]:
#visualizar datos
dataset_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [8]:
#visualizar tipo de datos
dataset_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
#verificar datos ausentes
dataset_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [10]:
#verificar datos duplicados en todo el dataset
print(dataset_0.duplicated().sum())

#verificar datos duplicados en los id
print(dataset_0['id'].duplicated().sum())

0
10


En el dataset 0 se visualizan tipos de datos correctos en cada columna y no hay datos ausentes. En cuanto a duplicados, en el dataset completo no encontré ninguno, sin embargo, al revisar la columna de IDs únicos, encontré 10 datos duplicados, los cuales voy a eliminar.

In [11]:
#eliminar datos duplicados de la columna ID
dataset_0 = dataset_0.drop_duplicates(subset='id')

#verificar nuevamente datos duplicados 
dataset_0['id'].duplicated().sum()


0

### 1.2.2. Preparar Dataset 1

In [12]:
#visualizar datos
dataset_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [13]:
#verificar informacion general del dataset
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
#verificar datos ausentes
dataset_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [15]:
#verificar datos duplicados en todo el dataset
print(dataset_1.duplicated().sum())

#verificar datos duplicados en los IDs
print(dataset_1['id'].duplicated().sum())

0
4


En el dataset 1 se visualizan tipos de datos correctos en cada columna y no hay datos ausentes. En cuanto a duplicados, en el dataset completo no encontré ninguno, sin embargo, al revisar la columna de IDs únicos, encontré 4 datos duplicados, los cuales voy a eliminar.

In [16]:
#eliminar datos duplicados de la columna ID
dataset_1 = dataset_1.drop_duplicates(subset='id')

#verificar nuevamente datos duplicados 
dataset_1['id'].duplicated().sum()


0

### 1.2.3. Preparar Dataset 2

In [17]:
#visualizar datos
dataset_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [18]:
#visualizar informacion general del dataset
dataset_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [19]:
#verificar datos ausentes
dataset_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [20]:
#verificar datos duplicados en el dataset completo
print(dataset_2.duplicated().sum())

#verificar datos duplicados en los IDs
print(dataset_2['id'].duplicated().sum())

0
4


En el dataset 2 se visualizan tipos de datos correctos en cada columna y no hay datos ausentes. En cuanto a duplicados, en el dataset completo no encontré ninguno, sin embargo, al revisar la columna de IDs únicos, encontré 4 datos duplicados, los cuales voy a eliminar.

In [21]:
#eliminar datos duplicados de la columna ID
dataset_2 = dataset_2.drop_duplicates(subset='id')

#verificar nuevamente datos duplicados 
dataset_2['id'].duplicated().sum()


0

Ya se encuentran los tres datasets preparados para comenzar a trabajar con ellos.

## 2. Modelo de regresión lineal

In [22]:
#definir variables para entrenar el modelo en el dataset 0
features_0 = dataset_0.drop(['id', 'product'], axis=1)
target_0 = dataset_0['product']

### 2.1. Segmentar el dataset 0 en conjuntos de entrenamiento y validación

In [23]:
#funcion para segmentacion de datos
def data_seg(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    return features_train, features_valid, target_train, target_valid

In [24]:
#segmentar el dataset 0 en conjuntos de entrenamiento y valiacion 75:25

features_train_0, features_valid_0, target_train_0, target_valid_0 = data_seg(features_0, target_0)

### 2.2. Entrenar el modelo y hacer predicciones para el conjunto de validacion

In [25]:
#entrenar el modelo 
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_valid_0 = model.predict(features_valid_0)

predictions_0 = pd.Series(predictions_valid_0)

### 2.3. Volumen medio de reservas predicho y RMSE del modelo

In [26]:

#calcular RMSE del modelo
mse_model = mean_squared_error(target_valid_0, predictions_0)
rmse_model = mse_model ** 0.05

print('Volúmen medio verdadero de reservas región "0":', target_valid_0.mean())
print('Volúmen medio de reservas predicho por el modelo:', predictions_0.mean())
print('RMSE del modelo:', rmse_model)
print()
print('Volúmen máximo de reserva:', target_valid_0.max())


Volúmen medio verdadero de reservas región "0": 92.15820490940044
Volúmen medio de reservas predicho por el modelo: 92.78915638280621
RMSE del modelo: 1.4381713586741687

Volúmen máximo de reserva: 185.33836970504785


### 2.4. Análisis de resultados


Luego de calcular el RMSE para las prediciones del modelo y obtener un valor de 1.43, me doy cuenta de que el modelo es bastante acertado considerando que estamos manejando un volumen máximo de reservas de hasta 185.33 miles de barriles. De modo que el modelo me parece lo bastante adecuado para la ejecución de la tarea.

### 2.5. Función para modelo de regresión lineal

In [107]:
#funcion para modelo de regresion lineal
def linear_regresion_model(features_train, features_valid, target_train, target_valid):

    #entrenar un modelo de regresion lineal 
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    #realizar predicciones con el conjunto de validacion
    predictions = model.predict(features_valid)
    predictions_valid = pd.Series(predictions)
    
    #crear un dataframe con target_valid y prediction_valid
    data_target_predictions = pd.DataFrame({'prediction_valid': predictions_valid, 'target_valid': target_valid})
    data_target_predictions = data_target_predictions.dropna()
    
    #separar el dataframe concatenado en dos variable
    predictions_to_revenue = data_target_predictions['prediction_valid']
    target_to_revenue = (data_target_predictions['target_valid'])[predictions_to_revenue.index]
    
    #calcular el promedio  de las predicciones del modelo
    predictions_mean = predictions_valid.mean()
    
    #calcular promedio de volumen verdadero de reservas
    target_valid_mean = target_valid.mean()
    
    #calcular RMSE del modelo
    mse_model = mean_squared_error(target_valid, predictions_valid)
    rmse_model = mse_model ** 0.05
    
    #crear variables con las metricas del modelo
    model_metrics = print('Volúmen medio verdadero de reservas :', target_valid_mean),
    print('Volúmen medio de reservas predicho por el modelo:', predictions_mean),
    print('RMSE del modelo:', rmse_model)
        
    return model_metrics, predictions_to_revenue, target_to_revenue

In [101]:
#definir variable en dataset 1
features_1 = dataset_1.drop(['id', 'product'], axis=1)
target_1 = dataset_1['product']

#segmentar datos para el dataset 1
features_train_1, features_valid_1, target_train_1, target_valid_1 = data_seg(features_1, target_1)

#entrenar modelo en dataset 1
model_metrics_1, predictions_to_revenue_1, target_to_revenue_1, data_1 = linear_regresion_model(features_train_1, features_valid_1, target_train_1, target_valid_1)

#sorted_values = trained_model_1.sort_values('prediction_valid', ascending=False).head(200)

#predicted_gain_1 = (sorted_values['prediction_valid'].sum()) * 4500 / 100000000

#target_gain_1 = (sorted_values['target_valid'].sum()) * 4500 / 100000000

print('metricas del modelo en la region "1"', model_metrics_1)
print()
print('prediciones', predictions_to_revenue_1)
print()
print('target', target_to_revenue_1)
print()
print('data', data_1)
 

Volúmen medio verdadero de reservas : 69.18604400957675
Volúmen medio de reservas predicho por el modelo: 69.17831957030432
RMSE del modelo: 0.988642715640864
metricas del modelo en la region "1" (None,)

prediciones 1        137.863458
2         29.745003
3         83.188229
8         83.083272
10        26.676041
            ...    
24977      2.440461
24980     30.532856
24983    108.679072
24987     29.395007
24991    135.653409
Name: prediction_valid, Length: 6146, dtype: float64

target 1         26.953261
2        134.766305
3        137.945408
8        134.766305
10        53.906522
            ...    
24977      0.000000
24980     84.038886
24983     84.038886
24987    137.945408
24991    134.766305
Name: target_valid, Length: 6146, dtype: float64

data        prediction_valid  target_valid
1            137.863458     26.953261
2             29.745003    134.766305
3             83.188229    137.945408
8             83.083272    134.766305
10            26.676041     53.906522

En el dataset 1 el modelo también es bantante preciso, con un RMSE de 0.98 en las predicciones.

In [49]:
#entrenando el modelo en dataset 2
features_2 = dataset_2.drop(['id', 'product'], axis=1)
target_2 = dataset_2['product']

predictions_2, predictions_mean_2, target_valid_mean_2, rmse_model_2 = linear_regresion_model(features_2, target_2)

print('Volúmen medio verdadero de reservas región "2":', target_valid_mean_2)
print('Volúmen medio de reservas predicho por el modelo:', predictions_mean_2)
print('RMSE del modelo:', rmse_model_2)

TypeError: linear_regresion_model() missing 2 required positional arguments: 'target_train' and 'target_valid'

Finalmente, en el dataset 2, el modelo da un RMSE de 1.44 en las predicciones. 
En los tres casos, el modelo erró entre 0.98 y 1.44. Sin embargo, me parece que el modelo es aceptable en los tres datasets ya que la diferencia no es un número demasiado alto, sobretodo cuando lo comparamos con los volúmenes de reserva máximos con los que estamos trabajando.

## 3. Cálculo de ganancias

In [86]:
#alamcacenar los valores para el cálculo en variables

inversion = 100000000
wells = 200
media_per_well = inversion / wells
min_units_per_well = media_per_well / 4500

In [87]:
#calcular la cantidad media de reservas en cada region
print('Unidades de reserva mínimas para evitar pérdidas en la inversión:', min_units_per_well)
print('Media de unidades de reserva en la región "0":', dataset_0['product'].mean())
print('Media de unidades de reserva en la región "1":', dataset_1['product'].mean())
print('Media de unidades de reserva en la región "2":', dataset_2['product'].mean())

Unidades de reserva mínimas para evitar pérdidas en la inversión: 111.11111111111111
Media de unidades de reserva en la región "0": 92.49968421774354
Media de unidades de reserva en la región "1": 68.82391591804064
Media de unidades de reserva en la región "2": 94.99834211933378


### Conclusiones

En promedio, ninguna de las zonas llega al promedio mínimo de reservas que se necesitan para no tener pérdidas con la inversión de 1 millón de dólares.

## 4. Función para el cálculo de las ganancias

In [109]:
#funcion para calcular las ganancias potenciales de los 200 principales pozos por region
def wells_revenue(target_to_revenue):
    
    #elegir los 200 pozos con predicciones más altas y sumar el volumen objetivo en cada region
    top_200_wells = target_to_revenue.sort_values(ascending=False).head(200)
    wells_volum_sum = top_200_wells.sum()

    #calcular la ganancia potencial en el target de los 200 pozos principales
    revenue = wells_volum_sum * 4500 - 100000000
    
    return revenue

In [110]:
#calcular las potenciales ganancias en cada region
#top_predicted_wells_0, gains_0 = well_gains(target_valid_0, predictions_0)
revenue_1 = wells_revenue(target_to_revenue_1)
#top_predicted_wells_2, gains_2 = well_gains(target_valid_2, predictions_2)

#print('Ganancias potenciales para los 200 principales pozos en la region "0":', gains_0)
print('Ganancias potenciales para los 200 principales pozos en la region "1":', revenue_1)
#print('Ganancias potenciales para los 200 principales pozos en la region "2":', gains_2)


Ganancias potenciales para los 200 principales pozos en la region "1": 24150866.966815114


En las tres regiones las ganancias resultan negativas. Propongo la region "0" en vista de que el número negativo es menor que en las otras regiones.

## 5. Calcular riesgos y ganancias para cada región

### 5.1. Utilizando las predicciones que almacenaste en 4.2, emplea el bootstrapping con 1000 muestras para hallar las distribucion de los beneficios.

In [50]:
#funcion para crear submuestras con bootstrapping
def bootstrap(predictions):
    
    state = np.random.RandomState(12345)
    values = []
    
    
    for i in range(1000):
        predicted_subsamples = predictions.sample(n=500, random_state=state, replace=True)
           
        values.append(predicted_subsamples)
    
    values = pd.Series(values)
    return values

## 5.2 Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje